### Generating Data 
This notebook goes over sampling methodology for our [recent work.](https://arxiv.org/abs/2407.05664), which mainly focusing on approximating composition chain of Sobolev functions.
To sample synthetic data, we used Matérn kernels whose smoothness and regularity properties closely aligns well with properties of Sobolev Space. Please check Corollary A.6 from [Tuo & Wu (2015)](https://arxiv.org/pdf/1508.07155)

The eq. for Matern kernel follows:
$$K(x_i, x_j) =  \frac{1}{\Gamma(\nu)2^{\nu-1}}\left(
         \frac{\sqrt{2\nu}}{l} d(x_i , x_j )
         \right)^\nu K_\nu\left(
         \frac{\sqrt{2\nu}}{l} d(x_i , x_j )\right)$$

As we mentioned in our paper on Appendix, Matérn kernel is considered as a generalization of RBF kernel. With the differentiability parameter $\nu$, it interpolates from 0 time differentiable kernel (Laplace Kernel) to infinitely differentiable or smooth kernel (Gaussian Kernel or RBF Kernel). The differentiability parameter $\nu$ is simply the number of differentiability of kernel + 0.5; that is, the Matérn kernel would be 2.3 times differentiable if $\nu=2.8$.

Notice that Matérn kernel enables us to find even non-integer times differentiable. Thanks to the modified Bessel function :D

In [5]:
#import packages 
from torch.distributions.multivariate_normal import MultivariateNormal
import torch

import pandas as pd
import numpy as np
import pickle
import itertools
import os 

from tqdm import tqdm
from MaternKernel import matern_Kernel

Our implementation `MaternKernel.py` is based on Matérn kernel implementation from `sklearn`, an open-source machine learning library. You may use `sklearn` instead of our code or even `torch`. Both libraries provide Matérn kernel implementation along with sampling components. We made that file just for brevity. Email me if you need either `sklearn` or `torch` version of this process.

However, we do not recommend using `gpytorch`, another open source library focusing on running Gaussian Process in PyTorch environment. I cannot doubt `gpytorch` is a great library, but it does not provide various range of diffrentiability: only 0, 1, 2. Remind that our work need to test differentiablility from 0.5 to infinity.

--------------------------------------------------------------------------------------------------
The following is the general procedure to sample data from Matérn kernel: 
1. Sample standard multivariate normal distribution $X \in \mathbb{R}^{N \times d_{in}}$
2. From $X$, compute the $N \times N$ kernel $K$ with given $\nu$
3. Sample $Y \in\mathbb{R}^{N \times d_{out}}$ with columns sampled from the Gaussian $\mathcal{N}(0,K)$.
 
where $N$ is the number of dataset; $d_{in}$ and $d_{in}$ are dimension of $X$ and $Y$, resp.

In [13]:
# initialize variables and sample GP X
N = 1000
d_in = 15
d_out = 20
nu = 15

X = np.random.randn(N, d_in) #1.
Y = matern_Kernel(X, nu = nu).sample(d_out) #2. & 3.

X.shape, Y.shape

01:06:21 AM:INFO:Sampling Matern Kernel with nu: 15
100%|██████████| 1000/1000 [00:00<00:00, 10484.29it/s]


((1000, 15), (1000, 20))

Done. Now all you have to do is to save `X` and `Y`. We saved each in pickle format with `N` and `nu`

But wait. Didn't we just said our paper focus on the composition of Sobolev functions? 
Yes, this is just a way to sample data from a **single** kernel. 

Now, we will find composition! Our new procedure for the composition of two Matérn kernel($K_g$, $K_h$) follows: 

1. Sample the training dataset $X \in \mathbb{R}^{N \times d_{in}}$   
2. From X, compute the $N \times N$ kernel $K_g$ with given $\nu_g$
3. From $K_g$, sample $Z \in\mathbb{R}^{N \times d_{mid}}$ with columns sampled from the Gaussian $\mathcal{N}(0,K_g)$.
4. From $Z$, compute $K_g$ with given $\nu_h$
5. From $K_h$, sample the test dataset $Y\in\mathbb{R}^{N \times d_{out}}$ with columns sampled from the Gaussian $\mathcal{N}(0,K_h)$.

----------------------------------------------------------------------------------------

Why only two kernels tho? It's because:
1. It would be easy to see track the trends of test error decay rate in 2D like Fig2. from our paper. 
3. `Y` from the composition of multiple random kernels would end up as stable state (constant). That is, all the data will be almost same each other.

In [ ]:
# initialize variables and sample GP X
full_N = 11500
d_in = 15 #input dimension for X
d_mid = 3 #dimension for the latent dataset Z
d_out = 20 #output dimension for Y

nu_g =  15 # differentiability of 1st kernel
nu_h =  2 # differentiability of 2nd kernel
############

X = np.random.randn(full_N, d_in)  
Z = matern_Kernel(X, nu = nu_g).sample(d_mid)
Y = matern_Kernel(Z, nu = nu_h).sample(d_out, Tikhonov = True)

pd.to_pickle(X, os.getcwd() + "/sampled_kernels/" + "X_g_Matern7_{nu_g}_{nu_h}_{N}.pkl".format(nu_g = nu_g, nu_h = nu_h, N=full_n))
pd.to_pickle(Y, os.getcwd() + "/sampled_kernels/" + "Y_g_Matern7_{nu_g}_{nu_h}_{N}.pkl".format(nu_g = nu_g, nu_h = nu_h, N=full_n))

Notice $\exists$ an argument `Tikhonov` in `Y`, but not in `Z`.

This `Tikhonov` means Tikhonov regularization, which add a very small constant to the diag($K$). 

Why do we need this? As we know, our computed kernels should be postive definite. **However**, precision issues in modern computer simply ignore very small numbers after certain floating point. This makes very similar two data points exactly same, which also leads linearly dependecy -> 0 eigvalues -> not positive definite -> cannot compute Matern kernel =[

Without this regularization, non positive definite kernels would like to occur more frequently. Notice `d_mid` is 3, which is more vulnerable to this issue when we try to compute `K_h`, the second Matérn kernel computed from `Z`. (bigger dimension of data is likely avoid those coincidence of having very very similar values)

This is the another reason that we only simulated the composition of two kernels. Compositioning more kernels would be more vulnerable to the non-positive definite issue due to the precision problem. 

--------------------------------------------------------------------------------------------------
Great! Now we need to sample all the combination of differentiability in the range [0.5,10]×[0.5,10]. The following code will sample 400 different combination of `X` and `Y`. Recall that due to time complexity and space complexity, running all the simulation would not be feasible for large $N$. This will take several hundreds of hours to finish, so we uploaded sampled kernels on Google Drive with more cases. Please download the kernels from our Drive instead of computing the almost exactly same task as we mentioned on our `Readme.md`for our 🌎

In [21]:
#differentiability [0.5,10]×[0.5,10]; 0.5 increment
full_N = 52500

#differentiability from 0.5 1.0 1.5 ... 19.5 20.0
num_dif = np.arange(20)/2 + 0.5

for n in list(itertools.product(num_dif, num_dif)):
    nu_g = 0.5 + n[0]
    nu_h = 0.5 + n[1]
    
    if not os.path.exists(os.getcwd() + "/sampled_kernels/" + "X_g_Matern7_{nu_g}_{nu_h}_50000.pkl".format(nu_g = n[0], nu_h = n[1])):
        df = pd.DataFrame()
        df.to_pickle(os.getcwd() + "/sampled_kernels/" + "X_g_Matern7_{nu_g}_{nu_h}_50000.pkl".format(nu_g = n[0], nu_h = n[1]))
        
        nu_g = 0.5 + n[0]
        nu_h = 0.5 + n[1]
        
        X = np.random.randn(full_N, d_in)  
        Z = matern_Kernel(X, nu = nu_g).sample(d_mid)
        try :
            Y = matern_Kernel(Z, nu = nu_h).sample(d_out)
        
        except:
            Y = matern_Kernel(Z, nu = nu_h).sample(d_out, Tikhonov = True)
        
        pd.to_pickle(X, os.getcwd() + "/sampled_kernels/" + "X_g_Matern7_{nu_g}_{nu_h}_50000.pkl".format(nu_g = n[0], nu_h = n[1]))
        pd.to_pickle(Y, os.getcwd() + "/sampled_kernels/" + "Y_g_Matern7_{nu_g}_{nu_h}_50000.pkl".format(nu_g = n[0], nu_h = n[1]))